In [53]:
VERSION = 4

In [58]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv("data/train.csv") 
print("Train shape:", train.shape )
TRAIN_LN = len(train)
display( train.head(1) )

test_df = pd.read_csv("data/test.csv") 
test_df['price'] = 0 # Doing this for common pre-processing steps.
print("Test shape:", test_df.shape )
display( test_df.head(1) )

Train shape: (188533, 13)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200


Test shape: (125690, 13)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes,0


In [59]:
CATS = [c for c in train.columns if not c in ["id","price"] ]
NUMS = ['milage']
CATS = [c for c in CATS if not c in NUMS]

In [60]:
test_df.drop(['price'], axis=1, inplace=True) # doing this for running inference on test data and fastai expects the column to not be present in the test data.

In [73]:
from fastai.tabular.all import *
from sklearn.model_selection import KFold
import torch

NUM_FOLDS = 5
RANDOM_STATE = 42
MAX_EPOCHS = 20
MODEL_CKPTS = f"checkpoints/version_{VERSION}"

# Check if MPS (Metal Performance Shaders) is available - MPS is only available on Macs with Apple Silicon.
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=RANDOM_STATE)
early_stop_callback = EarlyStoppingCallback(monitor='valid_loss', patience=3)
fold_rmse = []

test_predictions = np.zeros(len(test_df))

for fold, (train_idx, val_idx) in enumerate(kf.split(train), 1):
    val_fold = train.iloc[val_idx]
    dls_fold = TabularDataLoaders.from_df(train, y_names="price",
                                          cat_names=CATS,
                                          cont_names=NUMS,
                                          procs=[Categorify, FillMissing, Normalize],
                                          valid_idx=val_idx,
                                          device=device)
    
    learn = tabular_learner(dls_fold, metrics=rmse)
    learn.to(device)
    
    # TODO: from fastai.callback.tensorboard import TensorBoardCallback
    # TENSORBOARD_LOG_DIR=f"tensorboard_logs/version_{VERSION}/fold_{fold}"
    # tensorboard_callback = TensorBoardCallback(log_dir=TENSORBOARD_LOG_DIR)
    # learn.fit_one_cycle(EPOCHS, cbs=[tensorboard_callback]) 
    # Don't use tensorboard callback for some reason, it is throwing an error, fix this later.
    learn.fit_one_cycle(MAX_EPOCHS, cbs=[early_stop_callback])
    learn.save(MODEL_CKPTS+f"/fold_{fold}.pth")
    
    val_dl = learn.dls.test_dl(val_fold)
    preds, _ = learn.get_preds(dl=val_dl)
    fold_rmse.append(rmse(
        torch.tensor(preds.cpu().numpy() if isinstance(preds, torch.Tensor) else preds),
        torch.tensor(val_fold['price'].values)
    ))
    
    print(f"Fold {fold} RMSE: {fold_rmse[-1].item():.2f}")

    # Get test predictions for each fold
    test_dl = learn.dls.test_dl(test_df)
    fold_test_predictions = learn.get_preds(dl=test_dl)
    test_predictions += fold_test_predictions[0].cpu().numpy().reshape(-1,)

# Average the predictions across all folds
test_predictions /= NUM_FOLDS

print(f"\nMean RMSE across folds: {np.mean(fold_rmse):.2f}")
print(f"Standard deviation of RMSE: {np.std(fold_rmse):.2f}")

Fold 2


epoch,train_loss,valid_loss,_rmse,time
0,8303525888.000000,7514689536.000000,86687.320312,00:42
1,4832985600.000000,7503738880.000000,86624.125000,00:42
2,8703479808.000000,7456662016.000000,86351.953125,00:42
3,8098556416.000000,7304275968.000000,85465.054688,00:42
4,5334394368.000000,7003043840.000000,83684.171875,00:42
5,8833270784.000000,6643122688.000000,81505.343750,00:42
6,6041159680.000000,6180436992.000000,78615.750000,00:42
7,5640912384.000000,5816144896.000000,76263.664062,00:42
8,6184829440.000000,5495723008.000000,74133.148438,00:42
9,7196210688.000000,5181336576.000000,71981.492188,00:42


Fold 2 RMSE: 69027.81


Fold 3


epoch,train_loss,valid_loss,_rmse,time
0,10582999040.000000,8249888256.000000,90828.898438,00:42
1,5151597568.000000,8239401984.000000,90771.148438,00:42
2,6192506880.000000,8192693760.000000,90513.515625,00:42
3,7603979776.000000,8058383360.000000,89768.484375,00:40
4,6810397696.000000,7820807680.000000,88435.343750,00:41
5,6535215104.000000,7373929472.000000,85871.593750,00:41
6,4898815488.000000,6950466048.000000,83369.421875,00:41
7,9345357824.000000,6644310528.000000,81512.656250,00:41
8,2853265664.000000,6648114176.000000,81535.960938,00:42
9,6389785088.000000,6588287488.000000,81168.265625,00:42


...

Fold 5


epoch,train_loss,valid_loss,_rmse,time
0,5911291392.000000,8694432768.000000,93243.984375,00:42
1,5689995776.000000,8684393472.000000,93190.101562,00:43
2,5011401216.000000,8639185920.000000,92947.218750,00:42
3,4994456064.000000,8490302464.000000,92142.843750,00:42
4,7181341696.000000,8205058048.000000,90581.742188,00:42
5,6126220288.000000,7861803520.000000,88666.835938,00:43
6,8431433728.000000,7434397184.000000,86222.953125,00:43
7,5685537280.000000,6993481728.000000,83627.054688,00:42
8,5328416768.000000,6717258240.000000,81958.898438,00:42
9,6405854208.000000,6564970496.000000,81024.500000,00:42


No improvement since epoch 13: early stopping


Fold 5 RMSE: 78163.42



Mean RMSE across folds: 73803.54
Standard deviation of RMSE: 4023.47


In [74]:
test_df['price'] = test_predictions
test_df[['id','price']].to_csv(f'predictions/submission_v{VERSION}.csv',index=False)